# Import libraries

In [4]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

## ***Import database***

In [5]:
data = pd.read_excel("/home/poulami/Documents/temp/OnlineRetail.xlsx")

In [6]:
print(data.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


### [Breaktime](http://kissanime.ru/Anime/Steins-Gate/Episode-009?id=1507&s=default)

### ![picture](https://www.google.co.in/logos/doodles/2018/indias-republic-day-2018-5861462944776192-l.png)

In [7]:
data['total']=data.Quantity*data.UnitPrice
data['date']=data['InvoiceDate'].dt.date
print(data.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  total  \
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  15.30   
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  20.34   
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  22.00   
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  20.34   
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  20.34   

         date  
0  2010-12-01  
1  2010-12-01  
2  2010-12-01  
3  2010-12-01  
4  2010-12-01  


In [ ]:
stockcode = data[["StockCode"]]

In [ ]:
for x in stockcode['StockCode']:
    i1 = data[data['StockCode'] ==  data['StockCode'][0]][['InvoiceDate','Quantity','total','UnitPrice']]
    print(i1.head())